In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v1.5-beta0 (6/Dec/2022)


# Carbon Simulation - Demo 2-2

In this demo we look at **single-curve bidirectional liquidity ("strategies")**

Set up the simulation

In [2]:
Sim = CarbonSimulatorUI(verbose=False)
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair=None, mm='exact', xf=True)

We set up a single curve where the AMM sells ETH against USDC between 1500-2000, and buys it back at 1000-1500. The curve is seeded initially with 12247 USDC corresponding to 10 ETH at the effective price of the range.

In [3]:
Sim.add_strategy("ETH", 0, 1500, 2000, 1224, 1000, 1500, pair="ETH/USDC")
Sim

CarbonSimulatorUI(<2 orders, 0 trades>, pair=None, mm='exact', xf=True)

The warning about "swapping start and end" is because the selling obviously starts at 1500 USDC per ETH and goes all the way down to 1000 where it ends, so the numbers have been input incorrectly. The system can detect this an correct it, so now harm is done. Still, best practice is to add them in the right order, hence the warning.

In [4]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.0,0.0,ETH,1500.0,2000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,USDC,1224.0,1224.0,USDC,1500.0,1000.0,1500.0,USDC per ETH,0


## First cycle, part A: sell USDC

We first sell 224 USDC

In [5]:
Sim.amm_sells("USDC", 224, pair="ETH/USDC")["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,224.0,USDC,0.154523,ETH,ETHUSDC,1,1,1449.626512,USDC per ETH
0,0,0,A,AMM sells 224USDC buys 0ETH,True,True,None,224.0,USDC,0.154523,ETH,ETHUSDC,[1],1,1449.626512,USDC per ETH


In [6]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.154523,0.154523,ETH,1500.0,2000.0,1500.000000,USDC per ETH,1
1,1,ETHUSDC,USDC,1224.000000,1000.000000,USDC,1500.0,1000.0,1400.944684,USDC per ETH,0


And then we sell 1000 USDC

In [7]:
Sim.amm_sells("USDC", 1000, pair="ETH/USDC")["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #1,False,True,None,1000.0,USDC,0.844869,ETH,ETHUSDC,1,1,1183.615091,USDC per ETH
0,1,1,A,AMM sells 1000USDC buys 1ETH,True,True,None,1000.0,USDC,0.844869,ETH,ETHUSDC,[1],1,1183.615091,USDC per ETH


ETH is now empty, and USDC is loaded

In [8]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.999392,0.999392,ETH,1500.0,2000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,USDC,1224.000000,0.000000,USDC,1500.0,1000.0,1000.0,USDC per ETH,0


In [9]:
Sim.state()["trades"]

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,224.0,USDC,0.154523,ETH,ETHUSDC,1,1,1449.626512,USDC per ETH
0,0,0,A,AMM sells 224USDC buys 0ETH,True,True,None,224.0,USDC,0.154523,ETH,ETHUSDC,[1],1,1449.626512,USDC per ETH
0,1.0,1,0,route #1,False,True,None,1000.0,USDC,0.844869,ETH,ETHUSDC,1,1,1183.615091,USDC per ETH
0,1,1,A,AMM sells 1000USDC buys 1ETH,True,True,None,1000.0,USDC,0.844869,ETH,ETHUSDC,[1],1,1183.615091,USDC per ETH


In [10]:
p1 = Sim.state()["trades"].iloc[0]["price"]
p1

1449.626512

In [11]:
p2 = Sim.state()["trades"].iloc[2]["price"]
p2

1183.615091

In [12]:
print(f"Bought {224/p1 + 1000/p2} ETH at a price of {1224/(224/p1 + 1000/p2)}")

Bought 0.9993918151038297 ETH at a price of 1224.7448713324063


## First cycle, part B: sell ETH

In [13]:
Sim.amm_sells("ETH", 0.1, pair="ETH/USDC")

{'success': True,
 'trades':    uid  id subid                         note   aggr  exec limitfail  amt1  \
 0  2.0   2     0                     route #0  False  True      None   0.1   
 0    2   2     A  AMM sells 0ETH buys 152USDC   True  True      None   0.1   
 
   tkn1        amt2  tkn2     pair routeix  nroutes        price        p_unit  
 0  ETH  152.061848  USDC  ETHUSDC       0        1  1520.618484  USDC per ETH  
 0  ETH  152.061848  USDC  ETHUSDC     [0]        1  1520.618484  USDC per ETH  }

In [14]:
Sim.amm_sells("ETH", 0.9, pair="ETH/USDC")

{'success': False,
 'error': 'Insufficient liquidity across all user positions to support this trade.',
 'exception': AssertionError('Insufficient liquidity across all user positions to support this trade.')}

In [15]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,0.999392,0.899392,ETH,1500.0,2000.0,1541.040228,USDC per ETH,1
1,1,ETHUSDC,USDC,1224.000000,152.061848,USDC,1500.0,1000.0,1056.621274,USDC per ETH,0
